In [5]:
utils = require('jupyter-ijavascript-utils');
fs = require('fs');
['utils loaded']


[ 'utils loaded' ]

In [2]:
fs.writeFileSync("stage.css", `
#stage {
        position: sticky;
        left: 610px;
        top: 0;
        width: 800px;
        height: 500px;
        background: #000;
        border: 1px solid #ccc;
      }`);

In [4]:
utils.ijs.htmlScript({
    //-- automatically include a debugger statement
    debug: true,
    scripts: ["https://cdn.jsdelivr.net/gh/skulpt/skulpt-dist/skulpt.min.js",
    "https://cdn.jsdelivr.net/gh/skulpt/skulpt-dist/skulpt-stdlib.js",
    "https://cdn.bootcdn.net/ajax/libs/codemirror/5.59.2/codemirror.js",
    "https://cdn.bootcdn.net/ajax/libs/codemirror/5.59.2/mode/python/python.min.js",
    "https://cdn.jsdelivr.net/npm/skulpt-pygame-zero/dist/main.js",
             "https://unpkg.com/dexie/dist/dexie.js"],
    css: ["https://cdn.bootcdn.net/ajax/libs/codemirror/5.59.2/codemirror.css",
        "https://cdn.bootcdn.net/ajax/libs/codemirror/5.59.2/theme/idea.min.css",
         "./stage.css"],
    //-- initialize the root element with the following html
    html: `<form><div style="border: 1px solid #ccc; width: 800px;">
        <textarea id="yourcode" cols="80" rows="20">

</textarea>
      </div>
<br />
      <button type="button" id="run" style="width: 100px; height:30px; background:#f00; border:0; color:#fcf;">Save & Run</button>
    </form>
    <pre id="output"></pre>
    <div id="stage"></div>`,
    
    //-- size of the cell output size / html body (inner elements still need to be sized)
    width: 800,
    height: 600,
    
    //-- nodeJS functions we want to make available to JavaScript
    utilityFunctions: ({
        outf: (text) => {
        const mypre = rootEl.querySelector('pre#output');
        mypre.innerHTML = mypre.innerHTML + text;
      },
        getdb: () => {
            const db = new Dexie('pygame');
              db.version(1).stores({
                  data: ""
              });
            return db
        },
        builtinRead: (file) => {
        console.log("Attempting file: " + Sk.ffi.remapToJs(file));
        const externalLibs = {//"./numpy/__init__.js":'https://cdn.jsdelivr.net/gh/LIAO-wanting/skulpt_pj@latest/src/numpy_new/__init__.js',
            //"./numpy/__init__.js": "https://cdn.jsdelivr.net/gh/ebertmi/skulpt_numpy@master/numpy/__init__.js",
          //"./numpy/random/__init__.js": "https://raw.githubusercontent.com/ebertmi/skulpt_numpy/master/dist/numpy/random/__init__.js"
            "./numpy/__init__.js": 'https://raw.githubusercontent.com/waywaaard/skulpt_numpy/master/numpy/__init__.js',
            "./numpy/random/__init__.js":'https://raw.githubusercontent.com/waywaaard/skulpt_numpy/master/numpy/random/__init__.js'
                             };
        if (externalLibs[file] !== undefined) {
          return Sk.misceval.promiseToSuspension(
            fetch(externalLibs[file]).then(
              function (resp) { return resp.text(); }
            ));
        }
        
        if (Sk.builtinFiles === undefined || Sk.builtinFiles.files[file] === undefined) {
          throw "File not found: '" + file + "'";
        }
        
        return Sk.builtinFiles.files[file];
      }
        
    }),
    
    //-- data from nodeJS encoded and available to onReady
    data: {},
    
    //-- called when all scripts are loaded, and the html element is ready
    onReady: ({
        //-- the root element specific for this rendering
        rootEl,
        //-- the utilityFunctions parameter - nodeJS functions available in JavaScript
        utilityFunctions: lib,
        //-- the data parameter
        data,
        //-- an function that will render on the next animation frame
        animate
    }) => {const editor = CodeMirror.fromTextArea(rootEl.querySelector('textarea#yourcode'), {
        lineNumbers: true,
        mode:  "python",
        theme: "idea",
      });

      (Sk.TurtleGraphics || (Sk.TurtleGraphics = {})).target = 'stage';

      PyGameZero.setContainer(rootEl.querySelector('div#stage'))
        Promise.resolve().then(async () => {
            code = await lib.getdb().data.get('python');
            editor.setValue(code);
        }).catch(console.error);
        
        const mypre = rootEl.querySelector('pre#output');
        mypre.innerHTML = "";
        Sk.pre = "output";
        Sk.externalLibraries = {
        numpy : {
            path: 'https://raw.githubusercontent.com/waywaaard/skulpt_numpy/master/numpy/__init__.js'
        },
        'random': {
            path: 'https://github.com/nicoss01/python-editor-v5/raw/c46a87bbd2c206363acc694b77ca5edf92e24c68/libs/js/skulpt/src/lib/numpy/random/__init__.js'
        }
    };
        Sk.configure({
          output: lib.outf,
          read: PyGameZero.usePyGameZero(lib.builtinRead),
          __future__: Sk.python3,
        });
        rootEl.querySelector('button#run').onclick = function() {
          PyGameZero.reset();
            
          const prog = editor.getValue();
          const db = lib.getdb()
          Promise.resolve().then(async () => {
              await db.data.put(prog, 'python');
              console.log(await db.data.get('python'));
          }).catch(console.error);
          
          const myPromise = Sk.misceval.asyncToPromise(function() {
            return Sk.importMainWithBody("<stdin>", false, prog, true);
          });
          myPromise.then(
            function(mod) {
              console.log("success");
            },
            function(err) {
              console.error(err.nativeError);
              const mypre = rootEl.querySelector('pre#output');
              mypre.innerHTML = mypre.innerHTML + err;
            }
          );
        }
    }
})

Save & Run

In [1]:
game = `
from pgzrun import *
import random

WIDTH = 800
HEIGHT = 600
ship = Actor('https://s6.imgcdn.dev/Gm7Ol.png')
ship.x = 370
ship.y = 450
gem = Actor('https://s6.imgcdn.dev/GmBEh.png')
gem.x = random.randint(20, 780)
gem.y = 0
score = 0

def update():
    global score
    if keyboard.left:
        ship.x = ship.x - 5
    if keyboard.right:
        ship.x = ship.x + 5
    gem.y = gem.y + 4
    if gem.y > 600:
        gem.x = random.randint(20, 780)
        gem.y = 0
    if gem.colliderect(ship):
        gem.x = random.randint(20, 780)
        gem.y = 0
        score = score + 1

def draw():
    screen.fill((80,0,70))
    gem.draw()
    ship.draw()
    screen.draw.text('Pont:'+ str(score), (35,10), color='yellow', fontsize=30)
` ;

'\n' +
  'from pgzrun import *\n' +
  'import random\n' +
  '\n' +
  'WIDTH = 800\n' +
  'HEIGHT = 600\n' +
  "ship = Actor('https://s6.imgcdn.dev/Gm7Ol.png')\n" +
  'ship.x = 370\n' +
  'ship.y = 450\n' +
  "gem = Actor('https://s6.imgcdn.dev/GmBEh.png')\n" +
  'gem.x = random.randint(20, 780)\n' +
  'gem.y = 0\n' +
  'score = 0\n' +
  '\n' +
  'def update():\n' +
  '    global score\n' +
  '    if keyboard.left:\n' +
  '        ship.x = ship.x - 5\n' +
  '    if keyboard.right:\n' +
  '        ship.x = ship.x + 5\n' +
  '    gem.y = gem.y + 4\n' +
  '    if gem.y > 600:\n' +
  '        gem.x = random.randint(20, 780)\n' +
  '        gem.y = 0\n' +
  '    if gem.colliderect(ship):\n' +
  '        gem.x = random.randint(20, 780)\n' +
  '        gem.y = 0\n' +
  '        score = score + 1\n' +
  '\n' +
  'def draw():\n' +
  '    screen.fill((80,0,70))\n' +
  '    gem.draw()\n' +
  '    ship.draw()\n' +
  "    screen.draw.text('Pont:'+ str(score), (35,10), color='yellow', fontsize=30)\n"